# Capstone Project - The Battle of Neighborhoods
## Scenario: Brexit Relocation

### Introduction

In June 2016, the United Kingdom voted in a referendum to leave the European Union, which, as the details of the leaving process and future relationship got worked out, became bad news for financial services companies based in the UK as the trade agreement struck between the nation and the trading block has insufficient coverage for such companies to continue trading in the same way as they did before, leading to large shifts in capital and resources from the UK to the EU by companies in a move to ensure that access to the single market is not lost. <br><br>
The client, a financial services company with a substantial portfolio of EU assets under management, currently has its European headquarters in London and due to regulatory and market pressures has already had to follow suit and shift its asset ownership to EU country-registered entities. However, the client does not have an EU-based office – the EU entities it owns are legal/paper entities only - and so we finally arrive at the question we need to answer, which is “where should they move to”? <br><br>
This question, however, is too simple for its own good and as such requires discussion and breakdown. Following conversations with the client, the scope of what is to be delivered is as follows:<br><br>
•	They want to be provided a list (and presentation) of good landing spots as opposed to just one, so that the board can put it to a vote and decide<br>
•	They want as many staff to move with them as possible, and as such, candidate cities must be similar to London in both economic and cultural terms, and their distance to London must be considered as part of the exercise<br>
•	The top 50 EU urban areas by population are to be evaluated, disqualifying any which do not contain an international airport with links to London or a stock exchange<br><br>
As such, the chosen approach is a clustering model that will take in the requisite economic and cultural fit data and split the candidate cities (plus London) into groups, so that we can then extract the cities in the London cluster and present them as our list of viable options.


### Data

As discussed in the Introduction, the base data set to analyse is the top 50 EU urban areas by population, a list of which can be found <a href='https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_European_Union'>here</a>, plus London for comparison purposes. Where an urban area contains multiple cities, it is disambiguated into its component cities, which are only retained if their individual population is higher than the 50th ranked city in our data set.
This means the two composite areas on our list meet different fates - the Ruhr area, originally ranked 2nd, is disqualified (its largest city is Dortmund, which falls 140k short), and the only other composite urban area in the list, Rotterdam/The Hague, is split into two as both urban areas are comfortably in the top 50.<br>

Before assembling any further data, we must evaluate the disqualifying criteria and discard any cities that, per the requirements, do not have either an international airport serving its urban area or house a stock exchange – this leaves us with 28 cities in our preliminary data set.

In [8]:
import pandas as pd
prelim_df = pd.read_csv('cities_data_prelim.csv')
prelim_df.head()

,UrbanArea,Population,Density,IntlAirport,StockExchange
0,Paris,11020000,3800,Y,Y
1,Madrid,6026000,4600,Y,Y
2,Milan,4907000,2800,Y,Y
3,Barcelona,4588000,4300,Y,Y
4,Berlin,3972000,2900,Y,Y


In [10]:
prelim_df.shape

(28, 5)

Once the disqualified cities are removed, we must then locate and fill in the following to satisfy the economic side of our data requirements:
<ul>
    <li>Straight line distance to London</li>
    <li>Percentage of English speakers, according to <a href='https://ec.europa.eu/commfrontoffice/publicopinion/archives/ebs/ebs_386_en.pdf'>an EU Commission report on second languages</a> - assumed 100% in Ireland as English is official language</li>
    <li>Average commercial rental cost, sourced mostly from <a href='https://www.statista.com/statistics/431672/commercial-property-prime-rents-europe/'>this Statista dataset</a> (missing records searched individually)</li>
    <li>Average rental cost and income, sourced mostly from <a href='https://www.kaggle.com/stephenofarrell/cost-of-living'>this Kaggle dataset</a> (missing records searched individually at the source in Numbeo)</li>
    <li><a href='https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index'>HDI of country</a></li>
</ul>

For the cultural fit side, we will be extracting venue data from Foursquare for each city on the list, categorising venue types by popularity and appending the most common venue types to our data set for use by the clustering model. In order to prepare for this, we must also fetch coordinates data for our locations so that this can be fed into the API.<br>

The result will be a data set that looks like this:        

In [12]:
cities_df = pd.read_csv('cities_data.csv')
cities_df.head()

,UrbanArea,Population,Density,IntlAirport,StockExchange,DistanceToLdn,Lat,Long,HDI,PctFluentEng,CommRent,EmplRent,Income
0,Paris,11020000,3800,Y,Y,343.88,48.856697,2.351462,0.901,0.39,865.0,1177.51,2215.02
1,Madrid,6026000,4600,Y,Y,1264.80,40.416705,-3.703582,0.904,0.22,372.0,933.23,1578.77
2,Milan,4907000,2800,Y,Y,960.25,45.466800,9.190500,0.892,0.34,600.0,1099.67,1601.00
3,Barcelona,4588000,4300,Y,Y,1140.65,41.382894,2.177432,0.904,0.22,336.0,950.28,1444.48
4,Berlin,3972000,2900,Y,Y,931.48,52.517037,13.388860,0.947,0.56,468.0,896.50,2289.46
